In [2]:
pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine


/Users/abidhossain/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
import requests
import pandas as pd
import sqlalchemy
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def extract_with_retry(retries=3, timeout=10):
    API_URL = "http://universities.hipolabs.com/search?country=United+States"
    headers = {'User-Agent': 'Mozilla/5.0'}

    session = requests.Session()
    retry = Retry(
        total=retries,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
        raise_on_status=False
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    for attempt in range(retries):
        try:
            response = session.get(API_URL, headers=headers, timeout=timeout, stream=True)
            response.raise_for_status()

            # Read full content to avoid IncompleteRead
            full_data = b""
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    full_data += chunk
            return response.json() if full_data else []

        except (requests.exceptions.ChunkedEncodingError, requests.exceptions.IncompleteRead) as e:
            print(f"[Attempt {attempt+1}] Chunked/Incomplete Read error: {e}")
            time.sleep(2)
        except requests.exceptions.RequestException as e:
            print(f"[Attempt {attempt+1}] Request failed: {e}")
            time.sleep(2)

    raise Exception("Failed to fetch data after multiple retries")

def transform(data):
    return pd.json_normalize(data)

def load(df):
    engine = sqlalchemy.create_engine('sqlite:///universities.sqlite')
    df.to_sql('cal_uni', con=engine, if_exists='replace', index=False)
    print("✅ Data loaded into universities.sqlite")

# Main pipeline
if __name__ == "__main__":
    try:
        data = extract_with_retry()
        df = transform(data)
        load(df)
        print(df.head())
    except Exception as e:
        print("🚨 Pipeline failed:", e)


🚨 Pipeline failed: module 'requests.exceptions' has no attribute 'IncompleteRead'
